In [1]:
import json
import pandas as pd

In [2]:
# Step 1: Read the content of the first JSON file
with open('NER_TRAIN/NER_TRAIN_PREAMBLE.json', 'r') as file1:
    data = json.load(file1)

In [3]:
# Step 1: Read the content of the first JSON file
with open('NER_DEV/NER_DEV_PREAMBLE.json', 'r') as file1:
    data2 = json.load(file1)

In [4]:
len(data)

1560

In [ ]:
training_data = {'classes' : ['COURT', "PETITIONER", "RESPONDENT", "JUDGE", "LAWYER"], 'annotations' : []}
for example in data:
    temp_dict = {}
#     print(example['data']['text'])
#     ppp
    text = example['data']['text']
    temp_dict['text'] = text
#     print(temp_dict['text'])
    temp_dict['entities'] = []
#     print(example['annotations'][0]['result'])
#     ppp
    for annotation in example['annotations'][0]['result']:
        start = annotation['value']['start']
        end = annotation['value']['end']
        label = annotation['value']['labels'][0].upper()
        temp_dict['entities'].append((start, end, label))
    training_data['annotations'].append(temp_dict)
    
print(training_data['annotations'][156])

In [ ]:
test_data = {'classes' : ['COURT', "PETITIONER", "RESPONDENT", "JUDGE", "LAWYER"], 'annotations' : []}
for example in data2:
    temp_dict = {}
#     print(example['data']['text'])
#     ppp
    text = example['data']['text']
    temp_dict['text'] = text
#     print(temp_dict['text'])
    temp_dict['entities'] = []
#     print(example['annotations'][0]['result'])
#     ppp
    for annotation in example['annotations'][0]['result']:
        start = annotation['value']['start']
        end = annotation['value']['end']
        label = annotation['value']['labels'][0].upper()
        temp_dict['entities'].append((start, end, label))
    test_data['annotations'].append(temp_dict)
    
print(test_data['annotations'][15])

In [ ]:
!python -m spacy download en_core_web_sm

In [ ]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

# nlp = spacy.blank("en") # load a new spacy model
nlp = spacy.load("en_core_web_sm")
doc_bin = DocBin()

In [ ]:
from spacy.util import filter_spans

for training_example  in tqdm(training_data['annotations']): 
    text = training_example['text']
    labels = training_example['entities']
    doc = nlp.make_doc(text) 
    ents = []
    for start, end, label in labels:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    filtered_ents = filter_spans(ents)
    doc.ents = filtered_ents 
    doc_bin.add(doc)

doc_bin.to_disk("./train.spacy") # save the docbin object

In [ ]:
from spacy.util import filter_spans

for test_example in tqdm(test_data['annotations']): 
    text = test_example['text']
    labels = test_example['entities']
    doc = nlp.make_doc(text) 
    ents = []
    for start, end, label in labels:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    filtered_ents = filter_spans(ents)
    doc.ents = filtered_ents 
    doc_bin.add(doc)

doc_bin.to_disk("./dev.spacy") # save the docbin object

In [ ]:
!python -m spacy init fill-config base_config.cfg config.cfg

In [ ]:
!python -m spacy train config.cfg --output ./output --paths.train ./train.spacy --paths.dev ./dev.spacy